In [3]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
import numpy as np

In [4]:
# Đọc dữ liệu
review_data = pd.read_csv(r"C:\Users\anhn2\Documents\DJANGO\DA\TIKI\comments_data_mapped.csv")

# Chuẩn bị dữ liệu cho Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(review_data[['user_id', 'product_id', 'rating']], reader)

In [5]:

# Chia dữ liệu thành tập huấn luyện và kiểm tra
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
# Cấu hình các tham số cho GridSearch
param_grid = {
    'n_factors': [20, 50, 100],  # Số lượng yếu tố ẩn
    'reg_all': [0.02, 0.1, 0.2],  # Hệ số điều chỉnh
    'lr_all': [0.002, 0.005, 0.01]  # Tốc độ học
}

# Khởi tạo GridSearch
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)  # cv: số lần cross-validation
grid_search.fit(data)

# Lưu kết quả của từng bộ tham số vào DataFrame
results = []
for params, mean_rmse in zip(grid_search.cv_results['params'], grid_search.cv_results['mean_test_rmse']):
    results.append({
        'n_factors': params['n_factors'],
        'reg_all': params['reg_all'],
        'lr_all': params['lr_all'],
        'mean_rmse': mean_rmse
    })

results_df = pd.DataFrame(results)
results_df

,n_factors,reg_all,lr_all,mean_rmse
0,20,0.02,0.002,1.026722
1,20,0.02,0.005,1.018795
2,20,0.02,0.010,1.015965
3,20,0.10,0.002,1.027361
4,20,0.10,0.005,1.019007
5,20,0.10,0.010,1.014806
6,20,0.20,0.002,1.027913
7,20,0.20,0.005,1.019862
8,20,0.20,0.010,1.015776
9,50,0.02,0.002,1.027773


In [7]:
# Lựa chọn tham số tốt nhất và in chi tiết
best_params = grid_search.best_params['rmse']
best_rmse = grid_search.best_score['rmse']

print("\n--- Tham số tốt nhất ---")
print(best_params)
print(f"Best RMSE: {best_rmse}")


--- Tham số tốt nhất ---
{'n_factors': 20, 'reg_all': 0.1, 'lr_all': 0.01}
Best RMSE: 1.0148063794998348


In [8]:
# Khởi tạo mô hình với tham số tốt nhất
best_algo = SVD(n_factors=best_params['n_factors'], reg_all=best_params['reg_all'], lr_all=best_params['lr_all'])
best_algo.fit(trainset)
predictions = best_algo.test(testset)

# Đánh giá trên tập kiểm tra
final_rmse = accuracy.rmse(predictions)
print(f"Final RMSE on test set: {final_rmse}")

RMSE: 1.0044
Final RMSE on test set: 1.004389344155126


In [9]:
import pickle

# Lưu mô hình vào file
model_filename = "svd_model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_algo, model_file)
print(f"Mô hình đã được lưu vào file: {model_filename}")


Mô hình đã được lưu vào file: svd_model.pkl


In [10]:
from collections import defaultdict
from surprise import accuracy

# Hàm lấy Top-K sản phẩm được gợi ý cho mỗi người dùng
def get_top_k(predictions, k=10):
    top_k_recommendations = defaultdict(list)
    for prediction in predictions:
        uid = prediction.uid
        iid = prediction.iid
        est = prediction.est
        top_k_recommendations[uid].append((iid, est))
    # Sắp xếp theo giá trị dự đoán (est) giảm dần và lấy top K
    for uid, user_ratings in top_k_recommendations.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k_recommendations[uid] = user_ratings[:k]
    return top_k_recommendations

# Precision@K và Recall@K
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    # Lấy Top-K gợi ý
    top_k = get_top_k(predictions, k)
    precisions = []
    recalls = []

    for uid, user_ratings in top_k.items():
        # Các sản phẩm thực sự liên quan (n_rel)
        n_rel = sum((pred.r_ui >= threshold) for pred in predictions if pred.uid == uid)
        
        # Các sản phẩm được gợi ý trong Top-K (n_rec_k)
        n_rec_k = sum((est >= threshold) for (_, est) in user_ratings)
        
        # Các sản phẩm vừa liên quan vừa nằm trong Top-K (n_rel_and_rec_k)
        n_rel_and_rec_k = sum(
            ((pred.r_ui >= threshold) and (pred.est >= threshold))
            for pred in predictions
            if pred.uid == uid and any(iid == pred.iid for (iid, _) in user_ratings)
        )

        # Precision và Recall
        precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        precisions.append(precision)
        recalls.append(recall)

    # Trung bình Precision và Recall
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    return avg_precision, avg_recall

# Tính toán các giá trị Precision@K và Recall@K
precision_at_k, recall_at_k = precision_recall_at_k(predictions, k=10)

# Tính F1-Score
f1_score = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) != 0 else 0

# In kết quả
print(f"Precision@10: {precision_at_k:.4f}")
print(f"Recall@10: {recall_at_k:.4f}")
print(f"F1-Score@10: {f1_score:.4f}")


Precision@10: 0.8436
Recall@10: 0.8454
F1-Score@10: 0.8445


In [11]:
import time

# Lựa chọn tham số tốt nhất từ GridSearch
best_params = grid_search.best_params['rmse']

# Khởi tạo mô hình với tham số tốt nhất
best_algo = SVD(n_factors=best_params['n_factors'], reg_all=best_params['reg_all'], lr_all=best_params['lr_all'])

# Đo thời gian trước khi huấn luyện
training_start_time = time.time()

# Huấn luyện mô hình
best_algo.fit(trainset)

# Đo thời gian sau khi huấn luyện
training_end_time = time.time()

# Tính thời gian huấn luyện
training_time = training_end_time - training_start_time
print(f"Thời gian huấn luyện mô hình với tham số tốt nhất: {training_time:.2f} giây")


Thời gian huấn luyện mô hình với tham số tốt nhất: 0.03 giây
